# NLP Intent Recognition &mdash; Aufgaben

In diesem Notebook wollen wir den im Tutorial programmierten WetterBot erweitern. Dazu benötigen wir zunächst einige Deklarationen, Funktionen und Routinen aus dem Tutorial.

In [ ]:
import os
import numpy as np
import rasa_nlu.training_data
import rasa_nlu.config
from rasa_nlu.model import Trainer, Interpreter

from chatette.adapters import RasaAdapter
from chatette.parsing import Parser
from chatette.generator import Generator
import glob


DATA_DIR = 'data'
MODEL_DIR = 'models'
INTENTS_PATH = os.path.join(DATA_DIR, 'intents.md')
CHATETTE_DIR = os.path.join(DATA_DIR, 'chatette')
TRAIN_SAMPLES = 200


CONFIG_TF = """
language: de_core_news_sm
pipeline:
- name: "nlp_spacy"
  case_sensitive: true
- name: "tokenizer_spacy"
- name: "ner_crf"
- name: "ner_synonyms"
- name: "intent_featurizer_count_vectors"
- name: "intent_classifier_tensorflow_embedding"
"""


def write_file(filename, text):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(text)

        
def train(config=CONFIG_TF, intents_path=INTENTS_PATH):
    config_path = os.path.join(DATA_DIR, 'rasa_config.yml')
    write_file(config_path, config)
    trainer = Trainer(rasa_nlu.config.load(config_path))
    trainer.train(rasa_nlu.training_data.load_data(intents_path))
    return Interpreter.load(trainer.persist(MODEL_DIR))


def extract_intent(intent):
    return (intent['intent']['name'] if intent['intent'] else None,
            [(ent['entity'], ent['value']) for ent in intent['entities']])


def extract_confidences(intent):
    return (intent['intent']['confidence'] if intent['intent'] else None,
           [ent['confidence'] for ent in intent['entities']])


def test(interpreter, utterances):
    for utterance in utterances:
        intent = interpreter.parse(utterance)
        print('<', utterance)
        print('>', extract_intent(intent))
        print(' ', extract_confidences(intent))
        print()


def format_rules(rules, train_samples):
    train_str =  "('training':'{}')".format(train_samples)
    llines = [[name if (name[0] != '%') else name + train_str]
              + ['    ' + val for val in rules[name]] + [''] for name in rules]
    return '\n'.join((l for lines in llines for l in lines))


def chatette(rules, train_samples=TRAIN_SAMPLES):
    rules_path = os.path.join(DATA_DIR, 'intents.chatette')
    write_file(rules_path, format_rules(rules, train_samples))
    with open(rules_path, 'r') as rule_file:
        parser = Parser(rule_file)
        parser.parse()
    generator = Generator(parser)
    for f in glob.glob(os.path.join(CHATETTE_DIR, '*')):
        os.remove(f)
    RasaAdapter().write(CHATETTE_DIR, list(generator.generate_train()),
                        generator.get_entities_synonyms())

def generate_train_test(utterances, rules, config=CONFIG_TF):
    chatette(rules)
    test(train(config, CHATETTE_DIR), utterances)


Unser WetterBot versteht bisher einfache Fragen nach Temperaturen, Orten und Jahren. Wir wollen ihm nun beibringen, die Temperatur nicht nur zu einzeln angegebenen Jahren, sondern auch zu Zeitspannen zu verraten, also Angaben der Form

> 'von 1900 bis 2000'

oder

> 'zwischen 19777 und 1998'

zu verstehen. Schauen wir uns zum Einstieg nochmal die Regeln an, die wir im Tutorial verwendet hatten:

In [ ]:
RULES = {
    '@[Ort]': (
        'Brandenburg', 'Baden-Wuerttemberg', 'Bayern', 'Hessen',
        'Rheinland-Pfalz', 'Schleswig-Holstein', 'Saarland', 'Sachsen',
    ),
    '@[Zeit]': set(map(str, np.random.randint(1891, 2018, size=5))),
    '@[Komparativ]': set(['wärmer', 'kälter',]),
    '@[Superlativ]': set(['wärmsten', 'kältesten',]),
    '%[Frag_Temperatur]': set(['Wie {warm/kalt} war es ~[Zeit_Ort]',
                               'Welche Temperatur hatten wir ~[Zeit_Ort]',
                               'Wie war die Temperatur ~[Zeit_Ort]',]
    ),
    '%[Frag_Ort]': set([
        '~[wo_war] es @[Zeit] @[Komparativ] als {@[Zeit]/in @[Ort]}',
        '~[wo_war] es @[Zeit] am @[Superlativ]',]
    ),
    '%[Frag_Jahr]': set([
        '~[wann_war] es in @[Ort] @[Komparativ] als {@[Zeit]/in @[Ort]}',
        '~[wann_war] es in @[Ort] am @[Superlativ]',]
    ),
    '%[Ende]': set(['Ende', 'Auf Wiedersehen', 'Tschuess',]),
    '~[finde]': set(['Sag mir', 'Finde']),
    '~[wie_war]': set(['Wie war', '~[finde]',]),
    '~[was_war]': set(['Was war', '~[finde]',]),
    '~[wo_war]': set(['Wo war', 'In welchem {Bundesland|Land} war',]),
    '~[wann_war]': set(['Wann war', 'In welchem Jahr war',]),
    '~[Zeit_Ort]': set(['@[Zeit] in @[Ort]', '@[Ort] in @[Zeit]',]),
    '~[Bundesland]': set(['Land', 'Bundesland',]),
}




Die folgenden Testfragen sollten dann zum Beispiel richtig verstanden werden:

In [ ]:
TEST_UTTERANCES = ['Wie warm war es in Thüringen von 1990 bis 1996',
                   'Welche Temperatur hatten wir zwischen 1974 und 1980 im Saarland',
                   'Wie kalt war es 2010 und 2018 in Niedersachsen',
]

Insbesondere sollte bei der dritten Frage erkannt werden, dass keine Zeitspanne, sondern zwei einzelne Jahre gemeint sind. Mit den obigen Regeln werden diese Testfragen wie folgt verstanden:

In [ ]:
generate_train_test(TEST_UTTERANCES, RULES)


## Aufgabe 1: Zusätzliche Slots `Start` und `Ende` im Intent `Frag_Temperatur`

Ergänze `RULES` zu einem Python-Dictionary `RULES_A` durch

- neue Slots `@[Start]` und `@[Ende]`, die wie `@[Zeit]` Jahreszahlen als Beispiele haben,
- zusätzliche Beispiele für die Regel `~[Zeit_Ort]` mit Zeitspannen (und den neuen Slots)

und teste mit Hilfe von `generate_train_test`, ob die Test-Fragen dann richtig verstanden werden.


## Aufgabe 2: Ein neuer Intent `Frag_Temperatur_Spanne`

Ergänze alternativ nun die Regeln `RULES` um einen neuen Intent `Frag_Temperatur_Spanne` zu einem Dictionary `RULES_B`, ohne die Trainingsdaten für den Intent `Frag_Temperatur` zu beeinflussen. Teste wieder, ob die Test-Fragen so besser verstanden werden.

## Aufgabe 3: Zusätzliche Trainingsdaten für die Nennung zweier einzelner Jahre

Bei der dritten Testfrage wurden vielleicht in (b) die beiden einzelnen Jahre als Start und Ende einer Zeitspanne falsch verstanden. Können wir Abhilfe schaffen, indem wir solche Fragen in unsere Trainingsdaten aufnehmen? Teste das sowohl mit dem Ansatz aus Teilaufgabe (a) als auch mit dem Ansatz aus Teilaufgabe (b)!